In [19]:
# Start writing code here...
import numpy as np 
import pandas as pd
import collections
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
from tqdm.notebook import tqdm
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split
import torch.optim as optim
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

## Dataset

In [20]:
from string import punctuation
from collections import Counter 

def get_dset(dset_loc, tsteps):
    df = pd.read_csv(dset_loc)
    rev = []
    y = []
    lenseq = tsteps

    for i in range(0, 40000):
        rev.append(''.join([c for c in df.iloc[i][1].lower() if c not in punctuation]))
        y.append(df.iloc[i][0])

    all_text = ' '.join(rev)
    words = all_text.split()
    wcount = Counter(words)
    total_words = max(8000,len(words))
    sorted_words = wcount.most_common(total_words)
    vimap = {s[0]:i+1 for i, s in enumerate(sorted_words)}

    X = []
    for sent in rev:
        X.append([vimap.get(w, 0) for w in sent.split()])

    res = np.zeros((len(y), lenseq), dtype=int)
    for i in range(len(y)):
        inlen = len(X[i])
        if inlen <= lenseq:
            z = [0 for _ in range(lenseq - inlen)]
            r = z + X[i]
        else:
            r = X[i][:lenseq]
        res[i,:] = np.array(r)

    return res, np.array(y, dtype=int)-1, len(vimap)+1

In [21]:
dset_loc = "../input/yelp-review-dataset/yelp_review_polarity_csv/train.csv"

In [22]:
X, y, tksize = get_dset(dset_loc, 200)


In [23]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

print(X_train.shape, X_val.shape)

(32000, 200) (8000, 200)


In [24]:
from torch.utils.data import TensorDataset
batch_size = 128

train_data = TensorDataset(torch.from_numpy(X_train), torch.from_numpy(y_train))
val_data = TensorDataset(torch.from_numpy(X_val), torch.from_numpy(y_val))

train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
val_loader = DataLoader(val_data, shuffle=True, batch_size=batch_size)

In [25]:
diter = iter(train_loader)
testx, testy = next(diter)

print(testx.size(), testy.size())
print(testx, testy)

torch.Size([128, 200]) torch.Size([128])
tensor([[    0,     0,     0,  ...,   309,    10,  1682],
        [    0,     0,     0,  ...,   226,    33,    96],
        [   15,    31,    10,  ...,    80,   360,   125],
        ...,
        [    0,     0,     0,  ...,    51,  2994, 25340],
        [    0,     0,     0,  ...,    33,     3,  1160],
        [    0,     0,     0,  ..., 94857,  2207,    32]]) tensor([1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1,
        1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1,
        0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1,
        0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1,
        1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1,
        1, 0, 1, 1, 0, 1, 0, 1])


In [26]:
if torch.cuda.is_available():
    print("GPU")
    device = torch.device("cuda")
else:
    print("CPU")
    device = torch.device("cpu")

GPU



LSTM

In [27]:
from torch.autograd import Variable 

class LSTMClassifier(nn.Module):
    def __init__(self, ntoken, ninp, dim_hid, nlayers, 
                 nout, dropout=0.3):
        super().__init__()
        self.nhid = dim_hid
        self.nlayers = nlayers

        self.drop = nn.Dropout(dropout)

        self.embedding = nn.Embedding(ntoken, ninp)
        self.lstm = nn.LSTM(ninp, dim_hid, nlayers, 
                            dropout=dropout, 
                            batch_first=True)

        self.fc = nn.Linear(dim_hid, nout)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)
    
    def forward(self, x):
        h_0 = Variable(torch.zeros(self.nlayers, x.size(0), self.nhid)).to(device)
        c_0 = Variable(torch.zeros(self.nlayers, x.size(0), self.nhid)).to(device)
        # Propagate input through LSTM
        emb = self.embedding(x)
        output, (hn, cn) = self.lstm(emb, (h_0, c_0))
        hn = hn.view(-1, self.nhid)
        out = self.fc(hn)
        return out


In [28]:
from torchsummary import summary

gpu = torch.cuda.is_available()

model = LSTMClassifier(ntoken=tksize, ninp=18, dropout=0.3,
                       dim_hid=12, nlayers=1, nout=2)

summary(model, torch.zeros((1,200)).long())

Layer (type:depth-idx)                   Output Shape              Param #
├─Embedding: 1-1                         [-1, 200, 18]             1,888,434
├─LSTM: 1-2                              [-1, 200, 12]             1,536
├─Linear: 1-3                            [-1, 2]                   26
Total params: 1,889,996
Trainable params: 1,889,996
Non-trainable params: 0
Total mult-adds (M): 1.89
Input size (MB): 0.00
Forward/backward pass size (MB): 0.05
Params size (MB): 7.21
Estimated Total Size (MB): 7.26


Layer (type:depth-idx)                   Output Shape              Param #
├─Embedding: 1-1                         [-1, 200, 18]             1,888,434
├─LSTM: 1-2                              [-1, 200, 12]             1,536
├─Linear: 1-3                            [-1, 2]                   26
Total params: 1,889,996
Trainable params: 1,889,996
Non-trainable params: 0
Total mult-adds (M): 1.89
Input size (MB): 0.00
Forward/backward pass size (MB): 0.05
Params size (MB): 7.21
Estimated Total Size (MB): 7.26

In [29]:
lr = 0.01
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.RMSprop(model.parameters(), lr=lr)

epochs = 5
val_iters = len(val_loader)
if gpu:
    model.cuda()
model.train()

training_loss = []
training_acc = []
validation_loss = []
validation_accuracy = []
total_iters = len(train_loader)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, 0.5)

for e in range(epochs):
    print(f"epoch {e}")
    running_loss, training_correct, it = 0, 0, 0

    for (corpus, labels) in train_loader:
        it += 1

        if gpu:
            corpus, labels = corpus.type(torch.LongTensor).cuda(), labels.cuda()
        
        
        model.zero_grad()
        output = model(corpus)

        accuracy = (torch.argmax(output, axis=1) == labels.squeeze()).sum().item()
        training_correct += accuracy

        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    
    scheduler.step()
    training_loss.append(running_loss / it)
    training_acc.append(training_correct / len(train_data))

    model.eval()
    val_running_loss = 0
    val_correct = 0
    cnt = 0
    for (corpus, labels) in val_loader:
        cnt += 1
        if gpu:
            corpus, labels = corpus.type(torch.LongTensor).cuda(), labels.cuda()
        
        output = model(corpus)

        val_loss = criterion(output, labels)
        val_running_loss += val_loss.item()

        accuracy = (torch.argmax(output, axis=1) == labels.squeeze()).sum().item()
        val_correct += accuracy

    
    validation_loss.append(val_running_loss / cnt)
    validation_accuracy.append(val_correct / len(val_data))

    model.train()

    print(f"training loss {training_loss[-1]}")
    print(f"training accuracy {training_acc[-1]}")
    print(f"validation loss {validation_loss[-1]}")
    print(f"validation accuracy {validation_accuracy[-1]}")
    print(f"Current LR is {optimizer.param_groups[0]['lr']:.7f}", flush=True)
    print("-----------------------------------------------")

epoch 0
training loss 0.4696273545026779
training accuracy 0.78084375
validation loss 0.3734006134290544
validation accuracy 0.85
Current LR is 0.0050000
-----------------------------------------------
epoch 1
training loss 0.25839364421367644
training accuracy 0.90125
validation loss 0.2938916680831758
validation accuracy 0.8855
Current LR is 0.0025000
-----------------------------------------------
epoch 2
training loss 0.18740894338488578
training accuracy 0.93075
validation loss 0.2918812750823914
validation accuracy 0.893625
Current LR is 0.0012500
-----------------------------------------------
epoch 3
training loss 0.15292250816524028
training accuracy 0.9443125
validation loss 0.28910664977535366
validation accuracy 0.895
Current LR is 0.0006250
-----------------------------------------------
epoch 4
training loss 0.1377144345343113
training accuracy 0.9508125
validation loss 0.2940628015332752
validation accuracy 0.895375
Current LR is 0.0003125
-------------------------------